<a href="https://colab.research.google.com/github/RajeevPrakashAD1/ai_ml/blob/master/first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense,LeakyReLU,PReLU,ELU,Dropout,Activation
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Churn_Modelling.csv')
print(df.columns)
x = df.drop(['RowNumber','CustomerId','Surname','Exited'],axis='columns')
y = df['Exited']
print(x)


Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')
      CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0             619    France  Female   42       2       0.00              1   
1             608     Spain  Female   41       1   83807.86              1   
2             502    France  Female   42       8  159660.80              3   
3             699    France  Female   39       1       0.00              2   
4             850     Spain  Female   43       2  125510.82              1   
...           ...       ...     ...  ...     ...        ...            ...   
9995          771    France    Male   39       5       0.00              2   
9996          516    France    Male   35      10   57369.61              1   
9997          709    France  Female   36       7       0.00              

In [ ]:
#getting dummies variable for categorical
geo_dummy = pd.get_dummies(x["Geography"],drop_first='True')
gender_dummy = pd.get_dummies(x['Gender'],drop_first="True")
df1 = pd.concat([x,geo_dummy,gender_dummy],axis='columns')
df1.drop(["Geography","Gender"],axis="columns",inplace=True)

print(df1)

      CreditScore  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0             619   42       2       0.00              1          1   
1             608   41       1   83807.86              1          0   
2             502   42       8  159660.80              3          1   
3             699   39       1       0.00              2          0   
4             850   43       2  125510.82              1          1   
...           ...  ...     ...        ...            ...        ...   
9995          771   39       5       0.00              2          1   
9996          516   35      10   57369.61              1          1   
9997          709   36       7       0.00              1          0   
9998          772   42       3   75075.31              2          1   
9999          792   28       4  130142.79              1          1   

      IsActiveMember  EstimatedSalary  Germany  Spain  Male  
0                  1        101348.88        0      0     0  
1                  1   

In [ ]:
from sklearn.model_selection import train_test_split;
x_train,x_test,y_train,y_test = train_test_split(df1,y,test_size=0.2)
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)


In [ ]:
from pandas.core.common import standardize_mapping
#feature scalling
#basically it scales features and change the magnitude of input to a standaed scale
from sklearn.preprocessing import StandardScaler
x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)
# print(x_train,y_train)

In [ ]:
classifier = Sequential()
#classifier.add(Dense(output_dim=6,init="he_uniform",activation="relu",input_dim=11))
classifier.add(Dense(units=6,kernel_initializer="he_uniform",activation="relu",input_dim=11))
classifier.add(Dense(units=6,kernel_initializer="he_uniform",activation="relu"))
classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

#optimiser
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
                   
print(classifier.summary())


Model: "sequential_132"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_137 (Dense)           (None, 6)                 72        
                                                                 
 dense_138 (Dense)           (None, 6)                 42        
                                                                 
 dense_139 (Dense)           (None, 1)                 7         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model_history = classifier.fit(x_train,y_train,validation_split=0.33,batch_size=10,epochs=100)

Epoch 1/100
536/536 [==============================] - 3s 4ms/step - loss: 0.6153 - accuracy: 0.6824 - val_loss: 0.4726 - val_accuracy: 0.8080
Epoch 2/100
536/536 [==============================] - 2s 3ms/step - loss: 0.4582 - accuracy: 0.8028 - val_loss: 0.4416 - val_accuracy: 0.8118
Epoch 3/100
536/536 [==============================] - 2s 3ms/step - loss: 0.4327 - accuracy: 0.8138 - val_loss: 0.4269 - val_accuracy: 0.8217
Epoch 4/100
536/536 [==============================] - 2s 3ms/step - loss: 0.4234 - accuracy: 0.8145 - val_loss: 0.4200 - val_accuracy: 0.8251
Epoch 5/100
536/536 [==============================] - 2s 3ms/step - loss: 0.4181 - accuracy: 0.8168 - val_loss: 0.4141 - val_accuracy: 0.8281
Epoch 6/100
536/536 [==============================] - 2s 4ms/step - loss: 0.4143 - accuracy: 0.8205 - val_loss: 0.4085 - val_accuracy: 0.8319
Epoch 7/100
536/536 [==============================] - 2s 4ms/step - loss: 0.4102 - accuracy: 0.8233 - val_loss: 0.4037 - val_accuracy: 0.8387

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
y_pred = classifier.predict(x_test)
y_pred = (y_pred>0.5)
score=accuracy_score(y_pred,y_test)
print(score)

cm = confusion_matrix(y_test,y_pred)
print(cm)


63/63 [==============================] - 0s 2ms/step
0.8545
[[1505   76]
 [ 215  204]]


In [ ]:
from sklearn import metrics
from keras.layers.core.activation import Activation
def create_model(layers,activation):
  model = Sequential()
  for i,nodes in enumerate(layers):
    if i==0:
      model.add(Dense(nodes,input_dim=x_train.shape[1]))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(nodes))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
    
    model.add(Dense(units=1,kernel_initializer='glorot_uniform',activaiton='ReLU'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model
    

In [ ]:

# model = KerasClassifier(build_fn=create_model,verbose=0)

# layers = [[20],[40,20],[45,30,15]]
# activations=['relu','sigmoid']
# param_grid = dict(layers=layers,activation=activations,batch_size=[128,256],epochs=[30])
# grid = GridSearchCV(estimator=model,param_grid=param_grid,cv=5)
# grid_result = grid.fit(x_train,y_train)